# Use `.draw()` and `.from_draw()`

There are two ways to generate a sample of transients in ``skysurvey``, as shown in "Transients" documentation. We can use either `.draw()` or `.from_draw()`.

***

Although both produce simulated data, they differ in *what they return* and *how they modify the target object*.

## The instance method : `.draw()`

The `.draw()` method is used on an existing `Target` object instance (like snia or snii). It applies the simulation model to the parameters already defined within that object. When you already have a configured transient instance (for example, you set a magnitude distribution or added effects) and you want to simulate events following that model. It returns a pandas `DataFrame` of the simulated data (and updates `target.data` if `inplace=True`).

**Tip**:
- Parameters passed directly into `snii.draw(..., zmax=0.1, ...)` do not permanently change the underlying model of the snii instance. They are temporary settings for that specific draw call only. If you want a lasting change, use methods like `.set_magabs()`.

For example, let's create and configure a target instance (here a SNeII):

In [1]:
import skysurvey
snii = skysurvey.SNeII()
snii.set_magabs([-17.510, 0.959, 1.041])          

We get a sample of SNe II following the magabs distribution we set. Then, we draw 5000 SNe II and attach results to the instance:

In [ ]:
sim_data_ii = snii.draw(
    size=5_000,
    zmax=0.1,
    tstart='2018-04-01',
    tstop='2020-12-01',
    inplace=True # to store the simulated data in snii.data
)

**Tip**:
- If omitting `inplace=True`, the method will still return a `DataFrame` but snii.data will stay empty, meaning that it will not be possible to build a `DataSet` later using that instance.

The simulated data is stored directly in the instance such as:

In [3]:
snii.data.head()

,z,t0,magabs,ra,dec,magobs,template,amplitude
0,0.09445,58722.421875,-18.249115,196.688248,61.176659,20.006105,v19-asassn14jb-corr,3.219175e-15
1,0.05775,58599.359375,-15.904881,217.292450,22.320826,21.227455,v19-2016bkv-corr,1.045212e-15
2,0.08605,58779.089844,-17.599051,184.004181,-46.733185,20.441664,v19-1987a-corr,2.155366e-15
3,0.06505,59176.640625,-18.813171,73.091103,19.183865,18.588741,v19-2014g-corr,1.187655e-14
4,0.07325,58499.382812,-17.833073,25.852087,-23.146881,19.838968,v19-2009ib-corr,3.754905e-15


We can then create a `DataSet` using, for example, the ``ZTF`` survey.

In [ ]:

ztf_survey = skysurvey.ZTF.from_logs()
dset = skysurvey.DataSet.from_targets_and_survey(snii, ztf_survey)


## The class method : .from_draw()

`from_draw()` is a class method, meaning it builds a new transient object (a fresh instance) and immediately fills it with simulated data from the internal generative model defined by the transient class. 

This is convenient when wanting to use a ready-to-use object without manually creating or modifying one beforehand.

Let's create a new instance, drawn from the model:

In [ ]:
snii = skysurvey.SNeII.from_draw(
    size=5_000,
    zmax=0.1,
    tstart='2018-04-01', 
    tstop='2020-12-01'
)

So now snii is a `Target` object that already has snii.data populated.

In [6]:
snii.data.head()

,z,t0,magabs,ra,dec,magobs,template,amplitude
0,0.06575,58722.421875,-17.162199,82.332794,38.656788,20.264013,v19-2008in-corr,2.538522e-15
1,0.03195,58599.359375,-17.263792,159.456497,18.263813,18.542946,v19-2004et-corr,1.238820e-14
2,0.09835,58779.089844,-16.140957,358.779602,30.559660,22.207764,v19-2013ab-corr,4.237222e-16
3,0.09405,59176.640625,-16.249975,206.867401,24.577860,21.995449,v19-2008in-corr,5.152370e-16
4,0.07055,58499.382812,-17.949785,321.300659,47.007275,19.636662,v19-asassn15oz-corr,4.523995e-15


It is possible to directly build a `DataSet`:

In [7]:
dset = skysurvey.DataSet.from_targets_and_survey(snii, ztf_survey)

Warning : If you try to pre-configure an instance before calling `from_draw()` on it, those configurations are lost, as `from_draw()` creates an entirely new instance inside its logic.

The following will NOT work as expected (it will use the default SNe II model, not your configurations):

In [ ]:
# the configurations here are ignored by from_draw()
snii = skysurvey.SNeII()
snii.set_magabs([-17.510, 0.959, 1.041])
snii = skysurvey.SNeII.from_draw(...)
# this creates a new instance based on the base SNeII class defaults
# the previous snii configurations are lost

To apply custom parameters (like a magnitude distribution or an effect) using `from_draw()`, you must pass them in as part of the model argument. This ensures the new instance is created with the desired model configuration.

#### Step 1: Define custom model parameters:

In [ ]:
import skysurvey
import numpy as np 
RNG = np.random.default_rng()

custom_mag_model = {
    "magabs": {
        "func": RNG.normal,
        "kwargs": {"loc": -17.48, "scale": 0.7}, 
    },
}

#### Step 2: Create the new, simulated instance:

In [ ]:
snii = skysurvey.SNeII.from_draw(
    size=5_000,
    zmax=0.1,
    tstart='2018-04-01', 
    tstop='2020-12-01',
    model=custom_mag_model # your custom model
)

You can then obtain your `Dataset`:

In [ ]:
dset = skysurvey.DataSet.from_targets_and_survey(snii, ztf_survey)